# Program - 

**Purpose**


**Content**
- read data
- plot XY profiles

**Author:** Yi-Hsuan Chen (yihsuan@umich.edu)

**Date:** 

**Reference program:**



In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import io, os, sys, types

import yhc_module as yhc

xr.set_options(keep_attrs=True)  # keep attributes after xarray operation

## Read data

### TaiESM1 data

In [6]:
#--- datapath
datapath_TaiESM1 = "/Users/yi-hsuanchen/Downloads/yihsuan/research/data/TaiESM1/data/CMIP6/TaiESM1/amip-hist/atmos/mon/r1i1p1f1/"
fnames_rsut = [
    "rsut_Amon_TaiESM1_amip-hist_r1i1p1f1_gn_197901-201012.nc",
    "rsut_Amon_TaiESM1_amip-hist_r1i1p1f1_gn_201101-201412.nc",
]
fnames_rsut = [datapath_TaiESM1+"/"+file1 for file1 in fnames_rsut]

da_rsut = xr.open_mfdataset(fnames_rsut)


### CERES data

In [10]:
datapath_ceres = "/Users/yi-hsuanchen/Downloads/yihsuan/research/data/CERES/"
fname_ceres = datapath_ceres + "/" + "CERES_EBAF-TOA_Ed4.2_Subset_200101-201412.nc"

da_ceres = xr.open_dataset(fname_ceres)

## Data processing

## Plot - 

In [ ]:
#yhc.lib("fdef")   # check out yhc library